In [1]:
from skimage import feature
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
from joblib import dump, load
from sklearn import svm


In [2]:
# Function to calculate LBP feature
def get_lbp(image, num_points=24, radius=8):
    if len(image.shape) == 3 and image.shape[2] != 1:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Calculate LBP feature
    lbp = feature.local_binary_pattern(image, num_points, radius, method="uniform")
    
    # Calculate histogram
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, num_points + 3), range=(0, num_points + 2))
    
    # Normalize the histogram
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)
    
    return hist


def get_hog(image):
    # Initialize the HOG descriptor
    win_size = (192, 256)
    block_size = (128, 128)
    block_stride = (64, 64)
    cell_size = (64, 64)
    nbins = 9
    deriv_aperture = 1
    win_sigma = -1.
    histogram_normType = 0
    L2Hys_threshold = 0.2
    gamma_correction = 1
    n_levels = 9
    signed_gradient = True

    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins, deriv_aperture, win_sigma, histogram_normType, L2Hys_threshold, gamma_correction, n_levels, signed_gradient)

    # Compute HOG features
    features = hog.compute(image)

    return features

In [3]:
def get_image_shape(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    return image.shape

In [4]:
train_data_path = './train_img/train_img/color/'
test_data_path = './test_img/test_img/color/'

# load data
train_file_names = os.listdir(train_data_path)
test_file_names = os.listdir(test_data_path)
train_size = len(train_file_names)
test_size = len(test_file_names)

train_image_shape = get_image_shape(train_data_path + train_file_names[0])
test_image_shape = get_image_shape(train_data_path + train_file_names[0])

x_train = np.ndarray(shape=(train_size, train_image_shape[0], train_image_shape[1], train_image_shape[2]), dtype=np.uint8)
y_train = np.ndarray(shape=train_size, dtype=np.int8)
x_test = np.ndarray(shape=(test_size, test_image_shape[0], test_image_shape[1], test_image_shape[2]), dtype=np.uint8)
y_test = np.ndarray(shape=test_size, dtype=np.int8)

for i in range(train_size):
    image = cv2.imread(train_data_path + train_file_names[i], cv2.IMREAD_COLOR)
    x_train[i] = image

    if 'real' in train_file_names[i]:
        label = 0
    elif 'fake' in train_file_names[i]:
        label = 1

    y_train[i] = label 

for i in range(test_size):
    image = cv2.imread(test_data_path + test_file_names[i], cv2.IMREAD_COLOR)
    x_test[i] = image

    label = 1
    if 'real' in test_file_names[i]:
        label = 0
    y_test[i] = label

In [4]:
def get_feature_vector(image):
    lbp = get_lbp(image)
    hog = get_hog(image)

    feature_vector = np.concatenate((lbp, hog))
    return feature_vector

In [20]:
x_train_ = np.array(list(map(lambda img: get_feature_vector(img), x_train)))

In [21]:
x_test_ = np.array(list(map(lambda img: get_feature_vector(img), x_test)))

In [34]:
base_clf = svm.SVC(probability=True, kernel='rbf')

# Define the AdaBoost classifier
ada_clf = AdaBoostClassifier(estimator=base_clf, n_estimators=10, learning_rate=0.8, algorithm='SAMME.R', random_state=42)

# Train the AdaBoost classifier
ada_clf.fit(x_train_, y_train)
dump(ada_clf, 'ada_clf.joblib')

['ada_clf.joblib']

In [35]:
ada_pred = ada_clf.predict(x_test_)
ada_result = classification_report(ada_pred, y_test)
print(ada_result)

              precision    recall  f1-score   support

           0       0.81      0.91      0.86       524
           1       0.97      0.94      0.96      1884

    accuracy                           0.93      2408
   macro avg       0.89      0.92      0.91      2408
weighted avg       0.94      0.93      0.93      2408

